In [6]:
import requests
from urllib.request import urlopen
from urllib.parse import urlencode,unquote,quote_plus
import urllib
import lxml
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re

In [7]:
# 법령 XML 상세조회 (시행기준. target=eflaw)

#기본 url
url_base = "http://www.law.go.kr/DRF/lawService.do"
#조회 url 세부 설정
user_id = 'iambyunghyun' #open API ID (bmsong@kau.ac.kr의 아이디 부분)
target = "prec" #법령 조회시
ID = "220291" #ID 또는 MST #산업안전보건법 MST 218289
#ID = "001766" 
            #ID 산업안전보건법 법령ID 001766 (설명에는 ID로 조회하면 현행법령이 조회된다는데, 
            #target을 eflaw로 하면 ID로 요청해도, MST로 요청해도 같은 것 같다.
Type = "XML" #출력 형태 : HTML 또는 XML

url_sub = "?" +\
        "OC=" + user_id +\
        "&target=" + target +\
        "&ID=" + ID +\
        "&type=" + Type

#최종 url
url = url_base + url_sub

print(url)

response = requests.get(url)

#(1) response.text 사용하는 방법
#soup = BeautifulSoup(response.text.encode('utf-8'), 'lxml-xml') #lxml-xml -> lxml 추가 설치 필요(Beautifulsoup과 별도로 설치)

#(2) response.content 사용하는 방법
soup = BeautifulSoup(response.content, 'lxml-xml') #lxml-xml -> lxml 추가 설치 필요(Beautifulsoup과 별도로 설치)

soup

http://www.law.go.kr/DRF/lawService.do?OC=iambyunghyun&target=prec&ID=220291&type=XML


<?xml version="1.0" encoding="utf-8"?>
<PrecService><판례정보일련번호>220291</판례정보일련번호><사건명>추행</사건명><사건번호>2019도3047</사건번호><선고일자>20220421</선고일자><선고>선고</선고><법원명>대법원</법원명><법원종류코드>400201</법원종류코드><사건종류명>형사</사건종류명><사건종류코드>400102</사건종류코드><판결유형>전원합의체 판결</판결유형><판시사항>  [1] 동성인 군인 사이의 항문성교나 그 밖에 이와 유사한 행위가 사적 공간에서 자발적 의사 합치에 따라 이루어지는 등 군이라는 공동사회의 건전한 생활과 군기를 직접적·구체적으로 침해한 것으로 보기 어려운 경우, 군형법 제92조의6이 적용되는지 여부(소극)  [2] 군인인 피고인 甲은 자신의 독신자 숙소에서 군인 乙과 서로 키스, 구강성교나 항문성교를 하는 방법으로 추행하고, 군인인 피고인 丙은 자신의 독신자 숙소에서 동일한 방법으로 피고인 甲과 추행하였다고 하여 군형법 위반으로 기소된 사안에서, 피고인들과 乙은 모두 남성 군인으로 당시 피고인들의 독신자 숙소에서 휴일 또는 근무시간 이후에 자유로운 의사를 기초로 한 합의에 따라 항문성교나 그 밖의 성행위를 한 점 등에 비추어 피고인들의 행위는 군형법 제92조의6에서 처벌대상으로 규정한 ‘항문성교나 그 밖의 추행’에 해당하지 않는다고 한 사례
</판시사항><판결요지>  [1] [다수의견] 군형법 제92조의6의 문언, 개정 연혁, 보호법익과 헌법 규정을 비롯한 전체 법질서의 변화를 종합적으로 고려하면, 위 규정은 동성인 군인 사이의 항문성교나 그 밖에 이와 유사한 행위가 사적 공간에서 자발적 의사 합치에 따라 이루어지는 등 군이라는 공동사회의 건전한 생활과 군기를 직접적, 구체적으로 침해한 것으로 보기 어려운 경우에는 적용되지 않는다고 봄이 타당하다. 구체적인 이유는 다음과 같다.   (가) 현행 군형법 제92조의6은 2013. 4. 5. 법률 제11734호로 개정된 것

In [16]:
# API로 잘 받아들여와 지는지 확인용으로 있는것임

lc_basic_df = pd.DataFrame() #DataFrame for Law Content 

data = soup.find('사건명') #ID 정보
lc_basic_df['사건명'] = [data.get_text()]

data = soup.find('사건번호') 
lc_basic_df['사건번호']= [data.get_text()]

data = soup.find('선고일자') 
lc_basic_df['선고일자']= [data.get_text()]

data = soup.find('법원명')
lc_basic_df['법원명'] = [data.get_text()]

data = soup.find('법원종류코드') 
lc_basic_df['법원종류코드']= [data.get_text()]

data = soup.find('사건종류코드') 
lc_basic_df['사건종류코드']= [data.get_text()]

data = soup.find('판결유형') 
lc_basic_df['판결유형']= [data.get_text()]

lc_basic_df

,사건명,사건번호,선고일자,법원명,법원종류코드,사건종류코드,판결유형
0,추행,2019도3047,20220421,대법원,400201,400102,전원합의체 판결


In [9]:
판시사항_soup = BeautifulSoup(soup.판시사항.get_text(),'lxml')
# 판시사항 = re.sub('[0-9]',"",soup.판시사항.get_text().replace("<br>","").replace("<br/>","").replace('[',"").replace("]",""))
판시사항 = 판시사항_soup.get_text()
판시사항

'[1] 헌법상 보장되는 ‘변호인의 조력을 받을 권리’의 의미 및 피고인에게 국선변호인의 조력을 받을 권리를 보장하여야 할 국가의 의무에 국선변호인의 실질적 조력을 받을 수 있도록 할 의무가 포함되는지 여부(적극)[2] 공소사실 자체로 보아 어느 피고인에 대한 유리한 변론이 다른 피고인에게 불리한 결과를 초래하는 경우, 공동피고인들 사이에 이해가 상반되는지 여부(적극) / 이해가 상반된 피고인들 중 어느 피고인이 법무법인을 변호인으로 선임하고, 법무법인이 담당변호사를 지정하였는데 법원이 담당변호사 중 1인 또는 수인을 다른 피고인을 위한 국선변호인으로 선정한 경우, 국선변호인의 조력을 받을 피고인의 권리를 침해하는지 여부(적극)'

In [47]:
# 이게 형태가 좀 이상함 그리고 링크가 걸려있어서 이건 HTML로 끌고오는게 날지 생각해봐야함

판결요지_soup = BeautifulSoup(soup.판결요지.get_text(),'lxml')
판결요지 = 판결요지_soup.get_text()
판결요지

'[1] 헌법상 보장되는 ‘변호인의 조력을 받을 권리’는 변호인의 ‘충분한 조력’을 받을 권리를 의미하므로, 피고인에게 국선변호인의 조력을 받을 권리를 보장하여야 할 국가의 의무에는 피고인이 국선변호인의 실질적 조력을 받을 수 있도록 할 의무가 포함된다.[2] 공소사실 기재 자체로 보아 어느 피고인에 대한 유리한 변론이 다른 피고인에게는 불리한 결과를 초래하는 경우 공동피고인들 사이에 이해가 상반된다. 이해가 상반된 피고인들 중 어느 피고인이 법무법인을 변호인으로 선임하고, 법무법인이 담당변호사를 지정하였을 때, 법원이 담당변호사 중 1인 또는 수인을 다른 피고인을 위한 국선변호인으로 선정한다면, 국선변호인으로 선정된 변호사는 이해가 상반된 피고인들 모두에게 유리한 변론을 하기 어렵다. 결국 이로 인하여 다른 피고인은 국선변호인의 실질적 조력을 받을 수 없게 되고, 따라서 국선변호인 선정은 국선변호인의 조력을 받을 피고인의 권리를 침해하는 것이다.'

In [46]:
참조조문_soup = BeautifulSoup(soup.참조조문.get_text(),'lxml')
참조조문_list = []
for i in 참조조문_soup:
    try:
        참조조문_list.append(i.get_text())
    except:
        pass
참조조문_list

['[1] 헌법 제12조 제4항 / [2] 헌법 제12조 제4항, 형사소송법 제33조, 변호사법 제50조 제1항, 제7항']

In [45]:
참조판례_soup = BeautifulSoup(soup.참조판례.get_text(),'lxml')
참조판례_soup.get_text()

'[1] 대법원 2012. 2. 16.자 2009모1044 전원합의체 결정(공2012상, 480)'

In [8]:
판례내용_soup = BeautifulSoup(soup.판례내용.get_text())
판례내용 = 판례내용_soup.get_text()
판례내용

'【피 고 인】 【상 고 인】 피고인들【변 호 인】 변호사 임동언【원심판결】 서울서부지법 2015. 6. 11. 선고 2014노1566 판결【주    문】  원심판결 중 피고인 1에 대한 부분을 파기하고, 이 부분 사건을 서울서부지방법원 합의부에 환송한다.  피고인 2의 상고를 기각한다.【이    유】  상고이유(상고이유서 제출기간 경과 후에 제출된 상고이유보충서의 기재는 상고이유서를 보충하는 범위 내에서)를 판단한다.  1. 피고인 1의 상고이유에 대하여  가. 국선변호인의 실질적 조력을 받을 권리에 관한 법리헌법상 보장되는 ‘변호인의 조력을 받을 권리’는 변호인의 ‘충분한 조력’을 받을 권리를 의미하므로, 피고인에게 국선변호인의 조력을 받을 권리를 보장하여야 할 국가의 의무에는 피고인이 국선변호인의 실질적 조력을 받을 수 있도록 할 의무가 포함된다(대법원 2012. 2. 16.자 2009모1044 전원합의체 결정 등 참조).공소사실 기재 자체로 보아 어느 피고인에 대한 유리한 변론이 다른 피고인에 대하여는 불리한 결과를 초래하는 경우 공동피고인들 사이에 그 이해가 상반된다고 할 수 있다. 이와 같이 이해가 상반된 피고인들 중 어느 피고인이 특정 법무법인을 변호인으로 선임하고, 해당 법무법인이 담당변호사를 지정하였을 때, 법원이 위 담당변호사 중 1인 또는 수인을 다른 피고인을 위한 국선변호인으로 선정한다면, 국선변호인으로 선정된 변호사는 이해가 상반된 피고인들 모두에게 유리한 변론을 하기 어렵다. 결국 이로 인하여 위 다른 피고인은 국선변호인의 실질적 조력을 받을 수 없게 되었다고 보아야 하고, 따라서 위와 같은 국선변호인 선정은 국선변호인의 조력을 받을 피고인의 권리를 침해하는 것이다.  나. 피고인들의 이해 상반피고인 1에 대한 공소사실 중 피고인 2와 관련 있는 부분은, 피고인 1이 팔꿈치로 피고인 2의 가슴을 밀쳐 넘어뜨려 피고인 2에게 상해를 가하였다는 것이다.  한편, 피고인 2에 대한 공소사실은, 피고인 2가 위와 같이 상해를 당할 때 쓰레기통

In [9]:
판례목록_df = pd.read_html("법령검색목록.html")
판례목록_df = 판례목록_df[0]
# 판례목록_df
# 법령 XML 상세조회 (시행기준. target=eflaw)
info = []
def crawling_func(url):
    global info
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'lxml-xml') #lxml-xml -> lxml 추가 설치 필요(Beautifulsoup과 별도로 설치)
        판례내용_soup = BeautifulSoup(soup.판례내용.get_text())
        판례내용 = 판례내용_soup.get_text()
        info.append([ID,판례내용])
    except:
        print(url,"오류발생, 재귀호출 시작")
        time.sleep(1)
        crawling_func(url)
    finally:
        return

for idx,ID in enumerate(판례목록_df["판례ID"]):    
    #기본 url
    url = "http://www.law.go.kr/DRF/lawService.do?OC=bmsong&target=prec&ID={}&type=XML".format(ID)
    crawling_func(url)
    print(idx,ID)
    if idx ==10:
        break
    
info

/opt/anaconda3/lib/python3.8/site-packages/bs4/__init__.py:177: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 194 of the file /opt/anaconda3/lib/python3.8/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  warnings.warn(self.NO_PARSER_SPECIFIED_WARNING % dict(


0 216815
1 216821
2 216819
3 216807
4 216803
5 216801
6 216811
7 216813
8 216817
9 216809
10 216839


[[216815,
  '【원고, 피상고인】 원고 (소송대리인 변호사 방극성 외 2인)【피고, 상고인】 완주군수 (소송대리인 법무법인 가인로 담당변호사 강성명)【원심판결】 광주고법 2021. 2. 17. 선고 (전주)2020누1433 판결【주    문】원심판결을 파기하고, 사건을 광주고등법원에 환송한다.【이    유】상고이유를 판단한다.  1. 사안 개요  원심판결의 이유에 따르면 다음 사실을 알 수 있다.  가. 원고는 1994. 10. 17. 피고로부터 전북 완주군 (주소 생략) 지상 돼지 축사(이하 ‘이 사건 축사’라 한다)에 관하여「가축분뇨의 관리 및 이용에 관한 법률」(이하 ‘가축분뇨법’이라 한다)에 따라 배출시설 설치허가를 받고 이 사건 축사를 운영하고 있다.   나. 원고는 이 사건 축사에서 발생하는 가축분뇨를 아래 두 가지 방법을 혼용하여 처리해 왔다.  ① 완주군 공공처리시설에서 정화하여 방류하는 방법(이하 ‘공공처리시설 정화 후 방류 방법’이라 한다)   ② 이 사건 축사 부지에 설치된 액비(液肥) 자원화시설 등에서 액비로 만든 다음 인근 농경지 등에 살포하는 방법(이하 ‘액비화 살포 방법’이라 한다)   다. 원고는 2018. 12. 6. 피고에게 이 사건 축사 부지에 정화시설(이하 ‘이 사건 정화시설’이라 한다)을 설치하고 그곳에서 가축분뇨를 정화하여 방류하는 방법(이하 ‘자체 정화 후 방류 방법’이라 한다)으로 처리방법 변경허가 신청(이하 ‘이 사건 신청’이라 한다)을 하였다.   라. 피고는 2018. 12. 24. 원고에 대하여 다음과 같은 사유를 들어 처리방법 변경 불허가처분(이하 ‘이 사건 처분’이라 한다)을 하였다.   이 사건 신청은 공공처리시설과 자원화시설의 활용을 유도하는 정부의 정책에 부합하지 않고, 원고가 가축분뇨법령이 정한 방류수 수질기준을 준수할 것이라고 보이지도 않으며, 설령 수질기준을 준수한다고 하더라도 이 사건 정화시설의 방류수로 인해 새만금 수질 목표 달성에 지장을 초래할 수 있다. 특히 갈수기에는 오염물질이 

In [9]:
판례목록_df = pd.read_html("법령검색목록.html")
판례목록_df

[          순번    판례ID                                                 제목   법원  \
 0          1  216815                            가축분뇨배출시설변경허가신청불허가처분취소청구  대법원   
 1          2  216821                                               강도상해  대법원   
 2          3  216819                                 강제추행(인정된 죄명: 공연음란)  대법원   
 3          4  216807                                                대여금  대법원   
 4          5  216803                                            손해배상(기)  대법원   
 ...      ...     ...                                                ...  ...   
 39995  39996  103550                                               거절사정  대법원   
 39996  39997  103535                                               건물철거  대법원   
 39997  39998  103544                                              건물철거등  대법원   
 39998  39999  103581  공문서위조,공문서위조행사,사문서위조행사,공정증서원본불실기재행사,공정증서원본불실기재,...  대법원   
 39999  40000  103576                                교통사고처리특례법위반,도로교통법위반  대법원   
 
       사건유형  판결유형        선